In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import tensorflow as tf
import time
import tensorflow_datasets as tfds
import tensorflow_hub as hub

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import argparse
import os

from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Data Normalizers

In [ ]:
#Define labels and our dataframe

data = pd.DataFrame(columns=['image_path', 'label'])
labels = {
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask' : 'WithMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithMask' : 'WithMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithMask' : 'WithMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithoutMask' : 'NoMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithoutMask' : 'NoMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithoutMask' : 'NoMask'
}

In [ ]:
#Combining the training, test and validation sets into one dataframe

for folder in labels:
    for image_name in os.listdir(folder):
        image_path = os.path.join(folder, image_name)
        label = labels[folder]
        data = pd.concat([data, pd.DataFrame({'image_path': [image_path], 'label': [label]})], ignore_index=True)

In [ ]:
#Save the dataframe containing the whole labeled dataset as a .csv file in the working directory

data.to_csv('dataset.csv')

In [ ]:
#Reading the csv and converting it back to a dataframe

df = pd.read_csv("/kaggle/working/dataset.csv")
print(df)

In [ ]:
# Split into temporary set and testing set
temp_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Split the temporary set into training set and validation set (80-20)
train_df, val_df = train_test_split(temp_df, test_size=0.2, random_state=42)

In [2]:
Train_Normalized = ImageDataGenerator(rescale = 1./255.,rotation_range = 45, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)

Test_Normalized = ImageDataGenerator(rescale = 1.0/255.)

Validation_Normalized = ImageDataGenerator(rescale = 1.0/255.)

Importing and Normalizing the Dataset

In [3]:
train_gen = Train_Normalized.flow_from_dataframe(frame=train_df, x_col='image_path', y_col='label', target_size = (224,224), class_mode = 'categorical',  color_mode= 'rgb', batch_size = 64)

val_gen = Validation_Normalized.flow_from_dataframe(frame=val_df, x_col='image_path', y_col='label', target_size=(224,224), class_mode='categorical', color_mode= 'rgb', batch_size= 64)

test_gen = Test_Normalized.flow_from_dataframe(frame=test_df, x_col='image_path', y_col='label', target_size = (224,224), class_mode = 'categorical',color_mode= 'rgb', batch_size = 64)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [4]:
strategy = tf.distribute.MirroredStrategy()

In [10]:
def build_model():
    baseModel=MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))
    headModel=baseModel.output
    headModel=AveragePooling2D(pool_size=(7,7))(headModel)
    headModel=Flatten(name="flatten")(headModel)
    headModel=Dense(128,activation="relu")(headModel)
    headModel=Dropout(0.5)(headModel)
    headModel=Dense(2,activation="softmax")(headModel)
    model=Model(inputs=baseModel.input,outputs=headModel)
    for layer in baseModel.layers:
        layer.trainable=False
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])
    return model

In [11]:
with strategy.scope():
    model = build_model()
    
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']      

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=2,restore_best_weights=True, verbose=1)
history = model.fit(train_gen, epochs=20, validation_data=val_gen, callbacks=[early_stop])

Epoch 1/20
157/157 [==============================] - 232s 1s/step - loss: 0.6723 - accuracy: 0.6510 - val_loss: 0.3765 - val_accuracy: 0.8687
Epoch 2/20
157/157 [==============================] - 148s 940ms/step - loss: 0.4288 - accuracy: 0.8015 - val_loss: 0.2617 - val_accuracy: 0.9212
Epoch 3/20
157/157 [==============================] - 146s 933ms/step - loss: 0.3189 - accuracy: 0.8659 - val_loss: 0.2081 - val_accuracy: 0.9375
Epoch 4/20
157/157 [==============================] - 147s 934ms/step - loss: 0.2671 - accuracy: 0.8929 - val_loss: 0.1809 - val_accuracy: 0.9388
Epoch 5/20
157/157 [==============================] - 146s 933ms/step - loss: 0.2322 - accuracy: 0.9133 - val_loss: 0.1702 - val_accuracy: 0.9438
Epoch 6/20
157/157 [==============================] - 147s 936ms/step - loss: 0.2068 - accuracy: 0.9228 - val_loss: 0.1485 - val_accuracy: 0.9500
Epoch 7/20
157/157 [==============================] - 146s 930ms/step - loss: 0.1894 - accuracy: 0.9302 - val_loss: 0.1402 - va

In [13]:
results = model.evaluate(test_gen, verbose=0)
print("Test Loss: "+str(results[0]))
print("Test Accuracy: "+str(results[1]))

Test Loss: 0.06509368121623993
Test Accuracy: 0.9828628897666931


In [14]:
pred = model.predict(test_gen)
pred = np.argmax(pred,axis=1)

labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

16/16 [==============================] - 6s 234ms/step


In [15]:
model.save("Model/ask_mobileNetV2_detector.model", save_format="h5")